# Notebook Setup

In [66]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import timedelta

# pd.set_option('display.float_format', lambda x: '%.2f' % x) # suppress scientific notation
pd.set_option('display.max_rows', 150)

# Addresses and Desired Data

In [67]:
address_names = ['FP multisig','Phase3 HW']
addresses = ['0xbc49de68bcbd164574847a7ced47e7475179c76b','0x45e77E6473676aB30Ed64f3815Fea7847D02D7b8']
action_names = ['normal transactions', 'ERC20 transactions', 'NFT transactions']
actions = ['txlist','tokentx','tokennfttx']

# Requests

In [77]:
responses = []
dfs = []
for address in addresses:
    for action in actions:
        payload={
            'module':'account',
            'action':action,
            'address':address,
            'startblock':0,
            'endblock':'999999999',
            'sort':'asc',
            'apikey':'S53GR6DWMY1G9CY6X3U4Y9I9NKB8URMM5U'
        }
        r = requests.get('https://api.etherscan.io/api', params=payload)
        responses.append(json.loads(r.text))

for response in responses:
    dfs.append(pd.DataFrame.from_dict(response['result']))

In [90]:
df = dfs[4]
df = df[['timeStamp','from','to','tokenName','value']]
df = df.astype({'value': float})
df['value'] = df['value']/(10**18)
df = df[df['from'] != '0xbc49de68bcbd164574847a7ced47e7475179c76b']
df = df[df['tokenName'] == 'Fingerprints']

df

,timeStamp,from,to,tokenName,value
2,1628693679,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0x08f1b7bd6c6d9c6aea5b3a3b3225b953251d158d,Fingerprints,5000.00
4,1628695098,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0xc77fa6c05b4e472feee7c0f9b20e70c5bf33a99b,Fingerprints,5000.00
6,1628695473,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0x8ca9dff922d71f8a639d67b74a7f071f849b61d0,Fingerprints,5000.00
11,1628698574,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0xb1b28184e682f90de1e540291c0b9f88283e5d14,Fingerprints,5000.00
13,1628699176,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0x50664ede715e131f584d3e7eaabd7818bb20a068,Fingerprints,5000.00
15,1628699395,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0xd71a41ec000089ae99873afb4d15cc7d54dd95bc,Fingerprints,5000.00
17,1628699522,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0xa6775ae22067d71fae76e22fa377cfb38f223f0d,Fingerprints,5000.00
19,1628701232,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0x6924686b3dcd04ae8ef91a3feccc262d020f519d,Fingerprints,5000.00
23,1628704412,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0xa73dc2f955bca7f3f2d39c0926583174aa1ef0b4,Fingerprints,5000.00
25,1628707783,0x45e77e6473676ab30ed64f3815fea7847d02d7b8,0xec6d36a487d85cf562b7b8464ce8dc60637362ac,Fingerprints,5000.00


In [96]:
# Total prints traded so far
df['value'].sum()

198500.0